In [23]:
# STEP 1: CHOOSE AN ACTIVE REGION

# first import drms, aPython library for accessing HMI, AIA and MDI data from the 
# Joint Science Operations Center (JSOC) or other Data Record Management System (DRMS) servers
# see more here: https://docs.sunpy.org/projects/drms/en/stable/tutorial.html
import drms

# create a client for remote DRMS axess
c = drms.Client()

# choose a particular SHARP number
sharpnum = '377'

In [24]:
# STEP 2: SELECT DATA PRODUCTS TO DISPLAY

# choose particular keys and segments
user_key= 'HARPNUM,T_REC' 
user_seg= 'CONTINUUM, Br' 

# use  drms.client.Client.query() to obtain the data of all required keywords
keys, segments = c.query('hmi.sharp_cea_720s[' + sharpnum + '][]', key=user_key, seg=user_seg)

# obtain data from keys
harpnum, times= list(keys.HARPNUM)[0], list(keys.T_REC)

# obtain data from segments
Ic, Br= list(segments.CONTINUUM), list(segments.Br)

# these data cannot be accessed yet. They must be read in as fits files
# fits is a module for reading and writing Flexible Image Transport System (FITS) files.
from astropy.io import fits

# create arrays of data to visualize
# the key is to have approximately 100 data points for each movie
br = []
ic = []
step_time = []

# Determine the step size for selecting representative entries
step = max(len(Br) // 100, 1)

# Loop through the indices of Br and Ic with the defined step size
for i in range(0, len(Br), step):
    im_br = fits.open('http://jsoc.stanford.edu' + Br[i])
    br.append(im_br[1].data)
    im_br.close()

    im_ic = fits.open('http://jsoc.stanford.edu' + Ic[i])
    ic.append(im_ic[1].data)
    im_ic.close()

    step_time.append(times[i])

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

# Define the output file name
output_file = 'subplots_movie.gif'

# Create a list to store the cels
frames = []

# Iterate through the br and ic lists
for i in range(len(br)):
    # Create a figure with two subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Plot ic on the left subplot
    axs[0].imshow(ic[i], cmap='gray')
    axs[0].set_title('Continuum Intensity')

    # Plot br on the right subplot
    axs[1].imshow(br[i], cmap='gray')
    axs[1].set_title('Radial B Field')

    axs[0].text(0.05, 0.05, f'{step_time[i]}', color='white', fontsize=12,
                transform=axs[0].transAxes, ha='left', va='bottom', bbox=dict(facecolor='black', alpha=0.3))


    # Adjust the layout and spacing
    plt.tight_layout()

    # Save the current frame as an image
    plt.savefig(f'frame_{i}.png')

    # Close the figure to free up memory
    plt.close(fig)

    # Read the saved image and append to the frames list
    frames.append(imageio.imread(f'frame_{i}.png'))

# Save the frames as a GIF movie
imageio.mimsave(output_file, frames, duration=0.05)

# Delete saves images
for i in range(len(br)):
    filename = f'frame_{i}.png'
    os.remove(filename)